<a href="https://colab.research.google.com/github/choisanghoon/bme-haha/blob/main/capstone_1011.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np



import torch.optim as optim
from torch.utils.data import DataLoader
from matplotlib.pyplot import imshow 

import torchvision
import torchvision.transforms as transforms
from torchvision import transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

from matplotlib.pyplot import imshow 
# 이미지를 보여주기 위한 function을 불러옴
%matplotlib inline 
# matplotlib의 출력값이 노트북 상에 출력되게 함
# 불러온 이미지 파일이 Python 이미지 라이브러리를 통해 불러와지기 때문에 노트북 상에 표시되게 하기 위함


trans = transforms.Compose([
  transforms.Resize((224,224))                            
])
# torchvision의 transforms.Compose라는 function, transform을 많이 수행해야될 때 일일히 아래에 넣어주기 번거로워 하나로 묶어주는 역할

trans_data = torchvision.datasets.ImageFolder(root='/content/drive/My Drive/data set/chest_xray/train', transform=trans)

for num, value in enumerate(trans_data):
  data, label = value
  print(num, data, label)

  if (label == 0):
    data.save('drive/My Drive/Resize_train/NORMAL/%d_%d.jpeg'%(num, label))
  else:
    data.save('drive/My Drive/Resize_train/PNEUMONIA/%d_%d.jpeg'%(num, label))

In [ ]:
%matplotlib inline 

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()                             
])
trainset = torchvision.datasets.ImageFolder(root='drive/My Drive/Resize_train', transform=transform)
print(trainset)

train_data_mean = torch.Tensor([0.5, 0.5, 0.5])
train_data_std = torch.Tensor([0.5, 0.5, 0.5])
#train_data_mean = trainset.data.mean(axis=(0,1,2))
#train_data_std  = trainset.data.std(axis=(0,1,2))

print(train_data_mean)
print(train_data_std)

#train_data_mean = train_data_mean / 255
#train_data_std  = train_data_std / 255

print(train_data_mean)
print(train_data_std)

Dataset ImageFolder
    Number of datapoints: 5215
    Root location: drive/My Drive/Resize_train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )
tensor([0.5000, 0.5000, 0.5000])
tensor([0.5000, 0.5000, 0.5000])
tensor([0.5000, 0.5000, 0.5000])
tensor([0.5000, 0.5000, 0.5000])


In [ ]:
transform_train = transforms.Compose([
    transforms.Resize((300,300)),                      
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)                                  
])

transform_test = transforms.Compose([ 
    transforms.Resize((300,300)),                      
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean,train_data_std)                                  
])

trainset = torchvision.datasets.ImageFolder(root='drive/My Drive/Resize_train', transform=transform_train)
trainloader=torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)

testset = torchvision.datasets.ImageFolder(root='drive/My Drive/Resize_test', transform=transform_test)
testloader=torch.utils.data.DataLoader(testset, batch_size=16, shuffle=False, num_workers=4)

classes = ('NORMAL', 'PNEUMONIA')

---------------------- Resnet-50 ----------------------

In [ ]:
import torchvision.models.resnet as resnet

In [ ]:
conv1x1 = resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock = resnet.BasicBlock

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False, groups=1, width_per_group=64, replace_stride_with_dilation=None, norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None " "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)

        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

#        self.inplanes = 64
#        self.layer1 = self._make_layer(Bottleneck, 64, 3)
    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer

        downsample = None

        previous_dilation = self.dilation

        if dilate:
            self.dilation *= stride
            stride = 1

        if stride != 1 or self.inplanes != planes * block.expansion: # 64 != 256 {64 * 4(Bottleneck의 expansion)} 

        # downsample은 size가 다를 때 사용할 수 있다, 여기서는 channal을 맞추기 위해 사용한다
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),  # conv1x1(64, 64 * 4, stride = 1)
                norm_layer(planes * block.expansion), # nrom_layer(64 * 4)
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer)) # 레이어에 첫번째 저장
        # layers.append(Bottleneck(64, 64, stride = 1, downsample, self.groups, self.base_width, previous_dilation, norm_layer))

        self.inplanes = planes * block.expansion # self.inplanes = 64 * 4 = 256

        for _ in range(1, blocks):
        # layer1의 경우 for _ in range(1, 3): // 2번실행
            layers.append(block(self.inplanes, planes, groups=self.groups, base_width=self.base_width, dilation=self.dilation, norm_layer=norm_layer))

        return nn.Sequential(*layers)  # 순차적으로 진행하는 함수?

#  return되는 layers의 값 
#  self.layer1 =[
#  Bottleneck(64, 64, stride = 1, downsample, self.groups, self.base_width, previous_dilation, norm_layer)
#  Bottleneck(256, 64, groups=self.groups, base_width=self.base_width, dilation=self.dilation, norm_layer=norm_layer)
#  Bottleneck(256, 64, groups=self.groups, base_width=self.base_width, dilation=self.dilation, norm_layer=norm_layer)
#  ]

    def _forward_impl(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

In [ ]:
resnet50 = ResNet(resnet.Bottleneck, [3, 4, 6, 3], 10, True).to(device)

---------------------- end ----------------------

In [ ]:
# 동작확인
a=torch.Tensor(1,3,224,224).to(device)
out = resnet50(a)
print(out)

tensor([[-0.3410,  0.0053, -0.0515,  0.0178, -0.0672, -0.0869,  0.0526,  0.1925,
          0.2404, -0.4926]], device='cuda:0', grad_fn=<AddmmBackward>)


In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(resnet50.parameters(), lr = 0.1, momentum = 0.9, weight_decay=5e-4)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [ ]:
def test_acc_check(net, test_set, epoch, save=1):
  correct = 0
  total = 0
  with torch.no_grad():
    for data in test_set:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      outputs = net(images)

      _, predicted = torch.max(outputs.data, 1)

      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  test_acc = (100 * correct / total)
  print('Accuracy of the network on the 624 test images: %d %%' % test_acc)
#  if save:
#    torch.save(net.state_dict(). ".drive/My Drive/study/model/model_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
#  return acc

In [ ]:
def train_acc_check(net, train_set, epoch, save=1):
  correct = 0
  total = 0
  with torch.no_grad():
    for data in train_set:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      outputs = net(images)

      _, predicted = torch.max(outputs.data, 1)

      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  train_acc = (100 * correct / total)
  print('Accuracy of the network on the 5,216 train images: %d %%' % train_acc)
#  if save:
#    torch.save(net.state_dict(). ".drive/My Drive/study/model/model_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
#  return acc

In [ ]:
print(len(trainloader))
epochs = 50

for epoch in range(epochs):
  #optimizer.step() 파이썬 업데이트로 lr_sche.step() 이전에 optimizer.step()가 선언되어야한다고 오류떠서 앞으로 놔뒀더니 오류x
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    
    outputs = resnet50(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 30 == 29:
#        value_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader) ]))
        print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 30))
        running_loss = 0.0

  lr_sche.step()
  train_acc = train_acc_check(resnet50, trainloader, epoch, save=1)
  test_acc = test_acc_check(resnet50, testloader, epoch, save=1)
#  value_tracker(acc_plt, torch.Tensor([acc]), torch.Tensor([epoch]))

print('Finished Training')

326
[1,    30] loss: 0.063
[1,    60] loss: 0.042
[1,    90] loss: 0.042
[1,   120] loss: 0.036
[1,   150] loss: 0.061
[1,   180] loss: 0.044
[1,   210] loss: 0.087
[1,   240] loss: 0.065
[1,   270] loss: 0.041
[1,   300] loss: 0.055
Accuracy of the network on the 5,216 train images: 98 %
Accuracy of the network on the 624 test images: 70 %
[2,    30] loss: 0.030
[2,    60] loss: 0.036
[2,    90] loss: 0.032
[2,   120] loss: 0.032
[2,   150] loss: 0.031
[2,   180] loss: 0.036
[2,   210] loss: 0.027
[2,   240] loss: 0.053
[2,   270] loss: 0.024
[2,   300] loss: 0.039
Accuracy of the network on the 5,216 train images: 99 %
Accuracy of the network on the 624 test images: 71 %
[3,    30] loss: 0.011
[3,    60] loss: 0.017
[3,    90] loss: 0.026
[3,   120] loss: 0.042
[3,   150] loss: 0.014
[3,   180] loss: 0.024
[3,   210] loss: 0.028
[3,   240] loss: 0.014
[3,   270] loss: 0.030
[3,   300] loss: 0.020
Accuracy of the network on the 5,216 train images: 99 %
Accuracy of the network on the 6

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for data in testloader:
     images, labels = data
     images = images.to(device)
     labels = labels.to(device)
     outputs = resnet50(images)

     _, predicted = torch.max(outputs.data, 1)

     total += (predicted == labels).sum().item()

print('Accuracy of the network on the 624 test images: %d %%' %(
  100 * correct / total))

Accuracy of the network on the 624 test images: 0 %
